# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0823 12:44:44.386000 647732 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0823 12:44:44.386000 647732 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-23 12:44:45] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37135, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=47640

[2025-08-23 12:44:45] Using default HuggingFace chat template with detected content format: string


W0823 12:44:52.576000 648621 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0823 12:44:52.576000 648621 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0823 12:44:52.713000 648622 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0823 12:44:52.713000 648622 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-23 12:44:53] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-23 12:44:53] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=62.14 GB): 100%|██████████| 3/3 [00:00<00:00,  8.04it/s]
[2025-08-23 12:44:59] Capture cuda graph end. Time elapsed: 0.92 s. mem usage=0.07 GB. avail mem=62.13 GB.


[2025-08-23 12:45:00] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=62.13 GB


[2025-08-23 12:45:00] INFO:     Started server process [647732]
[2025-08-23 12:45:00] INFO:     Waiting for application startup.
[2025-08-23 12:45:00] INFO:     Application startup complete.
[2025-08-23 12:45:00] INFO:     Uvicorn running on http://0.0.0.0:37135 (Press CTRL+C to quit)
[2025-08-23 12:45:00] INFO:     127.0.0.1:60004 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-23 12:45:01] INFO:     127.0.0.1:60012 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-23 12:45:01] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:02] INFO:     127.0.0.1:60026 - "POST /generate HTTP/1.1" 200 OK
[2025-08-23 12:45:02] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-23 12:45:05] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:07] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.39, #queue-req: 0, 


[2025-08-23 12:45:07] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.76, #queue-req: 0, 


[2025-08-23 12:45:07] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.46, #queue-req: 0, 


[2025-08-23 12:45:08] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.87, #queue-req: 0, 


...

[2025-08-23 12:45:08] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.77, #queue-req: 0, 


[2025-08-23 12:45:09] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.71, #queue-req: 0, 


[2025-08-23 12:45:09] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.74, #queue-req: 0, 


[2025-08-23 12:45:09] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.75, #queue-req: 0, 


[2025-08-23 12:45:09] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 162.11, #queue-req: 0, 
[2025-08-23 12:45:09] INFO:     127.0.0.1:60162 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-23 12:45:09] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:10] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 150.11, #queue-req: 0, 


[2025-08-23 12:45:10] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.55, #queue-req: 0, 


[2025-08-23 12:45:10] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 170.14, #queue-req: 0, 


[2025-08-23 12:45:10] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.42, #queue-req: 0, 


[2025-08-23 12:45:11] Decode batch. #running-req: 1, #token: 216, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.64, #queue-req: 0, 


[2025-08-23 12:45:11] Decode batch. #running-req: 1, #token: 256, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.82, #queue-req: 0, 


[2025-08-23 12:45:11] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.10, #queue-req: 0, 


[2025-08-23 12:45:11] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 166.15, #queue-req: 0, 


[2025-08-23 12:45:12] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 167.23, #queue-req: 0, 
[2025-08-23 12:45:12] INFO:     127.0.0.1:60162 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-23 12:45:12] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:12] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 144.75, #queue-req: 0, 


[2025-08-23 12:45:12] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 168.06, #queue-req: 0, 


[2025-08-23 12:45:12] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.60, #queue-req: 0, 


[2025-08-23 12:45:13] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.22, #queue-req: 0, 


...

[2025-08-23 12:45:15] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.63, #queue-req: 0, 


[2025-08-23 12:45:15] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.61, #queue-req: 0, 


[2025-08-23 12:45:16] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, cuda graph: True, gen throughput (token/s): 128.90, #queue-req: 0, 


[2025-08-23 12:45:16] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.21, #queue-req: 0, 


[2025-08-23 12:45:16] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, cuda graph: True, gen throughput (token/s): 165.05, #queue-req: 0, 
[2025-08-23 12:45:16] INFO:     127.0.0.1:60162 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-23 12:45:16] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:16] Decode batch. #running-req: 1, #token: 31, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.37, #queue-req: 0, 


[2025-08-23 12:45:17] Decode batch. #running-req: 1, #token: 71, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.98, #queue-req: 0, 


[2025-08-23 12:45:17] Decode batch. #running-req: 1, #token: 111, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.96, #queue-req: 0, 
[2025-08-23 12:45:17] INFO:     127.0.0.1:60162 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-23 12:45:17] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:17] Decode batch. #running-req: 1, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 114.92, #queue-req: 0, 


[2025-08-23 12:45:17] Decode batch. #running-req: 1, #token: 531, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.76, #queue-req: 0, 


[2025-08-23 12:45:18] Decode batch. #running-req: 1, #token: 571, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.01, #queue-req: 0, 


[2025-08-23 12:45:18] Decode batch. #running-req: 1, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.82, #queue-req: 0, 


[2025-08-23 12:45:18] Decode batch. #running-req: 1, #token: 651, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.08, #queue-req: 0, 


[2025-08-23 12:45:18] Decode batch. #running-req: 1, #token: 691, token usage: 0.03, cuda graph: True, gen throughput (token/s): 155.80, #queue-req: 0, 


[2025-08-23 12:45:19] Decode batch. #running-req: 1, #token: 731, token usage: 0.04, cuda graph: True, gen throughput (token/s): 129.03, #queue-req: 0, 
[2025-08-23 12:45:19] INFO:     127.0.0.1:60162 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-23 12:45:21] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-23 12:45:21] Decode batch. #running-req: 1, #token: 51, token usage: 0.00, cuda graph: True, gen throughput (token/s): 15.11, #queue-req: 0, 


[2025-08-23 12:45:22] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, cuda graph: True, gen throughput (token/s): 169.82, #queue-req: 0, 


[2025-08-23 12:45:22] Decode batch. #running-req: 1, #token: 131, token usage: 0.01, cuda graph: True, gen throughput (token/s): 170.09, #queue-req: 0, 


[2025-08-23 12:45:22] Decode batch. #running-req: 1, #token: 171, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.54, #queue-req: 0, 


[2025-08-23 12:45:22] Decode batch. #running-req: 1, #token: 211, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.98, #queue-req: 0, 


[2025-08-23 12:45:23] Decode batch. #running-req: 1, #token: 251, token usage: 0.01, cuda graph: True, gen throughput (token/s): 159.33, #queue-req: 0, 


[2025-08-23 12:45:23] Decode batch. #running-req: 1, #token: 291, token usage: 0.01, cuda graph: True, gen throughput (token/s): 144.61, #queue-req: 0, 


[2025-08-23 12:45:23] Decode batch. #running-req: 1, #token: 331, token usage: 0.02, cuda graph: True, gen throughput (token/s): 145.87, #queue-req: 0, 


[2025-08-23 12:45:23] Decode batch. #running-req: 1, #token: 371, token usage: 0.02, cuda graph: True, gen throughput (token/s): 142.72, #queue-req: 0, 


[2025-08-23 12:45:24] Decode batch. #running-req: 1, #token: 411, token usage: 0.02, cuda graph: True, gen throughput (token/s): 159.71, #queue-req: 0, 
[2025-08-23 12:45:24] INFO:     127.0.0.1:56090 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-23 12:45:24] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:24] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: True, gen throughput (token/s): 157.74, #queue-req: 0, 


[2025-08-23 12:45:24] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: True, gen throughput (token/s): 156.43, #queue-req: 0, 


[2025-08-23 12:45:25] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: True, gen throughput (token/s): 89.58, #queue-req: 0, 


[2025-08-23 12:45:25] Decode batch. #running-req: 1, #token: 166, token usage: 0.01, cuda graph: True, gen throughput (token/s): 104.06, #queue-req: 0, 


...

[2025-08-23 12:45:26] Decode batch. #running-req: 1, #token: 246, token usage: 0.01, cuda graph: True, gen throughput (token/s): 82.22, #queue-req: 0, 


[2025-08-23 12:45:26] Decode batch. #running-req: 1, #token: 286, token usage: 0.01, cuda graph: True, gen throughput (token/s): 111.16, #queue-req: 0, 


[2025-08-23 12:45:27] Decode batch. #running-req: 1, #token: 326, token usage: 0.02, cuda graph: True, gen throughput (token/s): 167.77, #queue-req: 0, 


[2025-08-23 12:45:27] Decode batch. #running-req: 1, #token: 366, token usage: 0.02, cuda graph: True, gen throughput (token/s): 167.60, #queue-req: 0, 


[2025-08-23 12:45:27] Decode batch. #running-req: 1, #token: 406, token usage: 0.02, cuda graph: True, gen throughput (token/s): 167.92, #queue-req: 0, 
[2025-08-23 12:45:27] INFO:     127.0.0.1:48226 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-23 12:45:27] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-23 12:45:27] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-23 12:45:27] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0, 


[2025-08-23 12:45:27] Decode batch. #running-req: 3, #token: 122, token usage: 0.01, cuda graph: True, gen throughput (token/s): 316.15, #queue-req: 0, 


[2025-08-23 12:45:28] Decode batch. #running-req: 3, #token: 242, token usage: 0.01, cuda graph: True, gen throughput (token/s): 495.59, #queue-req: 0, 


[2025-08-23 12:45:28] Decode batch. #running-req: 3, #token: 362, token usage: 0.02, cuda graph: True, gen throughput (token/s): 498.15, #queue-req: 0, 


[2025-08-23 12:45:28] Decode batch. #running-req: 3, #token: 482, token usage: 0.02, cuda graph: True, gen throughput (token/s): 481.36, #queue-req: 0, 


[2025-08-23 12:45:28] Decode batch. #running-req: 1, #token: 208, token usage: 0.01, cuda graph: True, gen throughput (token/s): 418.54, #queue-req: 0, 


[2025-08-23 12:45:29] Decode batch. #running-req: 1, #token: 248, token usage: 0.01, cuda graph: True, gen throughput (token/s): 159.59, #queue-req: 0, 
[2025-08-23 12:45:29] INFO:     127.0.0.1:48230 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThe user has provided the correct information that the capital of France is Paris. However, the user's request was to provide the information of the capital of France, which is a bit ambiguous. It could be interpreted as wanting more detailed information about Paris, such as its history, culture, landmarks, or government. Alternatively, it might mean they want to confirm that Paris is indeed the capital, which they already know. \n\nTo better assist the user, it would be helpful to ask for clarification. If they want more details about Paris, I can provide a comprehensive overview. If they just need confirmation, then the current response suffices. \n\nAdditionally, I can offer to provide more information o

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-23 12:45:29] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:29] Decode batch. #running-req: 1, #token: 29, token usage: 0.00, cuda graph: True, gen throughput (token/s): 154.59, #queue-req: 0, 


[2025-08-23 12:45:29] Decode batch. #running-req: 1, #token: 69, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.61, #queue-req: 0, 


[2025-08-23 12:45:29] Decode batch. #running-req: 1, #token: 109, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.42, #queue-req: 0, 


[2025-08-23 12:45:30] Decode batch. #running-req: 1, #token: 149, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.90, #queue-req: 0, 


...

[2025-08-23 12:45:41] Decode batch. #running-req: 1, #token: 1869, token usage: 0.09, cuda graph: True, gen throughput (token/s): 156.28, #queue-req: 0, 


[2025-08-23 12:45:41] Decode batch. #running-req: 1, #token: 1909, token usage: 0.09, cuda graph: True, gen throughput (token/s): 167.30, #queue-req: 0, 


[2025-08-23 12:45:41] Decode batch. #running-req: 1, #token: 1949, token usage: 0.10, cuda graph: True, gen throughput (token/s): 166.80, #queue-req: 0, 


[2025-08-23 12:45:41] Decode batch. #running-req: 1, #token: 1989, token usage: 0.10, cuda graph: True, gen throughput (token/s): 163.90, #queue-req: 0, 


[2025-08-23 12:45:42] Decode batch. #running-req: 1, #token: 2029, token usage: 0.10, cuda graph: True, gen throughput (token/s): 164.39, #queue-req: 0, 
[2025-08-23 12:45:42] INFO:     127.0.0.1:48242 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-23 12:45:42] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-23 12:45:42] Decode batch. #running-req: 1, #token: 38, token usage: 0.00, cuda graph: True, gen throughput (token/s): 153.39, #queue-req: 0, 


[2025-08-23 12:45:42] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.33, #queue-req: 0, 


[2025-08-23 12:45:42] Decode batch. #running-req: 1, #token: 118, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.69, #queue-req: 0, 


[2025-08-23 12:45:43] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.05, #queue-req: 0, 


[2025-08-23 12:45:43] Decode batch. #running-req: 1, #token: 198, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.92, #queue-req: 0, 


[2025-08-23 12:45:43] Decode batch. #running-req: 1, #token: 238, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.33, #queue-req: 0, 
[2025-08-23 12:45:43] INFO:     127.0.0.1:54104 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0823 12:45:45.154000 646943 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0823 12:45:45.154000 646943 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0823 12:45:55.167000 651797 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0823 12:45:55.167000 651797 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.26s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.15s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.16s/it]



Capturing batches (bs=4 avail_mem=62.71 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=62.64 GB): 100%|██████████| 3/3 [00:00<00:00, 10.99it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. Let me know if you need more details.

That's the information I was looking for. Thank you!
Prompt: Give me the information of the capital of Germany.
Generated text: 
Alright, so I need to figure out the capital of Germany. Hmm, I'm pretty sure Germany has a capital city, but I can't remember exactly which one it is. Maybe I should think about the bigger cities in Germany. I know that Berlin is a major city there, but is it the capital?

Wait, I think so. I've heard people talking about Berlin as the capital because it's a significant political and economic hub. But I'm not 100% certain. Maybe I should try to recall any other major cities in Germany. Then there's Munich, which is also a big city, but I
Prompt: Give me the information of the capital of Italy.
Generated text: 
The capital of Italy is Rome.

The statement is incorrect because the capital of Italy is actually Rome, b

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its history, culture, economy, and tourism.

In 20 words or less, describe each of the following: Your response must be in English, and consist of four separate sentences, each describing one of the key aspects (history, culture, economy, tourism) of London.

Additionally, write a short paragraph (3-5 sentences) about London that ties these aspects together, and ends with a statement about why London is considered a major global city.

The entire response must be in English, with no markdown, bullet points, or bold text.

Alright, I need to help the user by gathering information about London as a major global
Prompt: Please provide information about Paris as a major global city:
Generated text:  its history, culture, economy, and geography.

2. Provide a detailed description of the Eiffel Tower, including its construction, design, materials, and significance.

3. Discuss the impact of the COVID-19 

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to figure out what exactly they're asking for. They want the JSON format, which is a way to structure data. 

The capital of France is Paris, so that's straightforward. Now, they also want the population, so I need to provide that number. I should make sure it's the most recent figure. From what I remember, the population of Paris has been increasing, so I think it's around 2 million. But I should double-check that to be accurate.

I wonder if the user is a student working on a project or maybe someone planning a visit. They might need this data for a report or a presentation. Since they specified JSON, they're probably looking for structured data that's easy to integ

In [19]:
llm.shutdown()